# VGGish Pretrained

In [136]:
import numpy as np

In [137]:
human1_deconv = np.load('data/LivingRoom_preprocessed_hack/Human1/deconvoled_trim.npy')
human1_deconv

array([[[ 9.07240337e-06,  1.71257434e-05,  1.22597794e-05, ...,
          8.86178623e-06,  1.40686043e-05,  1.06616626e-05],
        [-8.23853679e-06, -8.23373648e-06, -8.74563466e-06, ...,
          2.87394778e-06, -8.90485808e-06, -3.52464713e-06],
        [ 6.62780906e-07, -1.40201587e-06,  1.83228190e-06, ...,
         -1.06218545e-06,  8.53497477e-06,  2.28155182e-06],
        [ 8.11119719e-07, -2.47451499e-06,  5.82140547e-06, ...,
          7.73804004e-06,  2.29692650e-05,  1.10566034e-05]],

       [[ 1.64466037e-05,  1.53698293e-05,  1.91599720e-05, ...,
          1.76548729e-05,  1.70955263e-05,  1.41169767e-05],
        [ 1.05826217e-07,  9.13145777e-06,  1.76140716e-07, ...,
          1.21369967e-05,  1.55958387e-05,  1.23088676e-05],
        [ 7.06836090e-06,  3.26818258e-06,  7.15508941e-06, ...,
          4.64980394e-06,  8.96262554e-06,  6.55734584e-06],
        [-3.18089551e-05, -8.93297238e-06, -2.51026504e-05, ...,
         -7.02741545e-07,  1.34268867e-05,  3.24307

In [138]:
human1_deconv.shape

(1000, 4, 667200)

In [139]:
human1_centroid = np.load('data/LivingRoom_preprocessed_hack/Human1/centroid.npy')
human1_centroid

array([[-3231.3293467 , -1127.87771457],
       [-3198.54107875,  -744.5100656 ],
       [-3192.9776274 ,  -248.26678827],
       ...,
       [-1717.89923578, -3166.59648491],
       [-1808.60337549, -2779.13038427],
       [   44.43741322,   106.48353609]])

In [140]:
human1_centroid.shape

(1000, 2)

In [141]:
def normalize_centroid(centroid):
    _max = np.array([500, 2000])
    _min = np.array([-4000, -4000])
    return 2 * ((centroid - _min) / (_max - _min)) - 1

human1_centroid_norm = normalize_centroid(human1_centroid)

In [142]:
from sklearn.model_selection import train_test_split
import torch

X_train, X_test, y_train, y_test = train_test_split(human1_deconv, human1_centroid_norm, test_size=0.2, random_state=42)

# precut_off = 92580

# cutting the sound before the cut
# X_train = X_train[..., :precut_off]
# X_test = X_test[..., :precut_off]

X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)

y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)

In [144]:
import os
from pathlib import Path
import sys

# Add external libraries
sys.path.insert(0, '/home/onyxia/work/hackathon-acoustic-2024/external')

# LIBS_DIR = Path(os.path.join(os.getcwd(),'/home/onyxia/work/hackathon-acoustic-2024/external/torchvggish_main'))

In [145]:
import torch
from torchvggish_main.vggish import VGG, VGGish, make_layers
from torchvggish_main.vggish_input import waveform_to_examples
import torchvggish_main.mel_features
import torchvggish_main.vggish_params




# TODO(sclarke): This was just copied from the hubconf file in the original repo, this should use an external reference.
vggish_model_urls = {
    'vggish': 'https://github.com/harritaylor/torchvggish/'
              'releases/download/v0.1/vggish-10086976.pth',
    'pca': 'https://github.com/harritaylor/torchvggish/'
           'releases/download/v0.1/vggish_pca_params-970ea276.pth'
}

def vggish(pretrained=True, **kwargs):
    model = VGGish(urls=vggish_model_urls, pretrained_net=pretrained, postprocess=pretrained, **kwargs)
    return model

class VGGModel(torch.nn.Module):
        
        def __init__(self, fs=16000, pretrained=True):
            super(VGGModel, self).__init__()
            # self.vggish = torch.hub.load('harritaylor/torchvggish', 'vggish')
            self.vggish = vggish(pretrained=pretrained)
            self.vggish.eval()
            self.fs = fs
            
        def forward(self, x):
            x = (self.vggish.forward(torch.unsqueeze(x, axis=1), fs=self.fs) / 127) - 1
            # x = (self.vggish.forward(np.expand_dims(x, axis=1), fs=self.fs) / 127) - 1
            return x    
    
def get_finetune_model(pretrained=True, frozen=True, out_channels=2, dropout=False, in_channels=1):
    # TODO(sclarke): This pretrained model is based on 16kHz samples, and automatically downsamples input audio to 16k
    vggish_model = VGGModel(fs=int(16e3), pretrained=pretrained)
    if pretrained and frozen:
        for param in vggish_model.parameters():
            param.requires_grad = False
    p_dropout = 0.5 if dropout else 0.0
    finetune_model = torch.nn.Sequential(
        vggish_model,
        torch.nn.Linear(in_features=128, out_features=256, bias=True),
        torch.nn.ReLU(),
        torch.nn.Dropout(p_dropout),
        torch.nn.Linear(in_features=256, out_features=256, bias=True),
        torch.nn.ReLU(),
        torch.nn.Dropout(p_dropout),
        # torch.nn.Linear(in_features=256, out_features=256, bias=True),
        # torch.nn.ReLU(),
        torch.nn.Linear(in_features=256, out_features=out_channels, bias=True)
    )
    
    return finetune_model

In [146]:
net = get_finetune_model(in_channels=4) # in_channels useful?
xy_loss_fn = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(params=net.parameters(), lr=1e-4, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.999995)

In [115]:
vggish_cutoff = 30590

X_train = X_train[..., :vggish_cutoff]
X_test = X_test[..., :vggish_cutoff]

In [147]:
X_train.shape

torch.Size([800, 4, 667200])

### Passage Multi Channel vers Single Channel

In [148]:
X_train = torch.mean(X_train, dim = 1)
X_test = torch.mean(X_test, dim = 1)

In [149]:
X_train.shape
X_test.shape

torch.Size([200, 667200])

In [150]:
norm_val_min = np.min(human1_deconv)
norm_val_range = np.max(human1_deconv) - norm_val_min
epsilon = 1e-2

def postprocess_net_output(output):
    output[:, :2] = norm_val_range * ((torch.tanh(output[:, :2]) * (1 + epsilon)) + 1) / 2 + norm_val_min
    return output

In [155]:
import numpy as np

batch_size = 32

N_iter = int(X_train.shape[0] / batch_size)
epochs = 100

train_losses = []
train_xy_losses = []
valid_losses = []
valid_xy_losses = []
step_count = 0

for n in range(epochs):
    print(f'Reshuffling and starting Epoch {n+1}/{epochs}', flush=True)
    rand_idx = np.random.permutation(X_train.shape[0])
    net.train()
    optimizer.zero_grad()
    epoch_train_loss = 0  # To calculate average loss per epoch
    for i in range(N_iter):
        curr_idx = rand_idx[i * batch_size : (i + 1) * batch_size]
        print(curr_idx)
        print(X_train.shape)
        print(X_train[curr_idx, :])
        net_out = net(X_train[curr_idx, :])
        results = postprocess_net_output(net_out)
        xy_loss = xy_loss_fn(results[:, :2], y_train[curr_idx, :2])
        loss = xy_loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        train_loss = loss.item()
        train_losses.append((step_count, train_loss))
        train_xy_losses.append((step_count, xy_loss.item()))
        epoch_train_loss += train_loss
        
        if i % 50 == 0:  # Adjust the frequency of the print statement as needed
            print(f"Epoch {n+1}, Step {i+1}/{N_iter}, Current Batch Loss: {train_loss:.4f}", flush=True)

        step_count += 1

    average_epoch_loss = epoch_train_loss / N_iter
    print("===========================================")
    print(f"Epoch {n+1} completed. Average Loss: {average_epoch_loss:.4f}", flush=True)
    print("===========================================")
    #TODO Validation
    # valid_loss_xy_arr = np.zeros(valid_waves.shape[0], dtype=np.float32)
    # valid_loss_arr = np.zeros(valid_waves.shape[0], dtype=np.float32)
    # for i in range(valid_waves.shape[0]):
    #     with torch.no_grad():
    #         results = torch.squeeze(postprocess_net_output(net(torch.unsqueeze(valid_waves[i, :], axis=0)).view(-1, 1)))
    #     xy_loss = xy_loss_fn(results[:2], valid_xy[i, :2])
    #     valid_loss_xy_arr[i] = xy_loss.item()
    #     loss = xy_loss
    #     valid_loss_arr[i] = loss.item()
    # valid_xy_loss = np.mean(valid_loss_xy_arr)
    # valid_loss = np.mean(valid_loss_arr)
    # print('Validation XY Loss: %0.3f'%valid_xy_loss)
    # print('Validation Loss: %0.3f'%valid_loss)
    # valid_losses.append((step_count, valid_loss))
    # valid_xy_losses.append((step_count, valid_xy_loss))


Reshuffling and starting Epoch 1/100
[ 42 262 154 645 709  16 265 293 231 114 657 682 392 712  94 271  99 380
 767 407 621 234 748 292 473 158 629 539 728 540 577 762]
torch.Size([800, 667200])
tensor([[-4.0744e-06, -3.9708e-06, -2.1449e-06,  ..., -3.4175e-06,
         -3.9267e-06, -8.7405e-07],
        [ 1.9350e-05,  8.0068e-06, -1.7974e-05,  ...,  4.4959e-07,
          5.1767e-06,  5.8309e-06],
        [-1.1513e-05, -1.2718e-05, -4.2134e-06,  ..., -5.5409e-06,
         -9.7882e-06, -6.1775e-06],
        ...,
        [ 1.3555e-05,  1.1145e-05,  1.1900e-05,  ...,  8.8881e-06,
          1.3050e-05,  8.6300e-06],
        [-1.1412e-06,  7.1517e-06, -1.3595e-06,  ...,  4.2832e-06,
          5.1993e-06,  5.3835e-06],
        [ 5.6862e-06,  8.2866e-06,  6.3170e-06,  ...,  3.8902e-06,
          5.2080e-06,  3.1029e-06]])
Epoch 1, Step 1/25, Current Batch Loss: 0.2201
[691 472  39 653 331 780 516  15 771 213 451 207 544  31 405 547 272  74
 749 656  73 570 515 128 311 530 731 532 217 175 190 1

In [156]:
first_spectogram

tensor([ 5.5560e-06,  2.8784e-06,  2.5103e-06,  ..., -2.7700e-05,
         4.2482e-06,  2.5084e-05])

In [157]:
net.eval()  # Set the model to evaluation mode

_max = np.array([500, 2000])
_min = np.array([-4000, -4000])

distances = []

for i in range(len(X_test)):
    first_spectrogram, first_actual_coordinate = X_test[i], y_test[i]
    first_spectrogram = torch.tensor(first_spectrogram, dtype=torch.float32).unsqueeze(0)

    with torch.no_grad():
        predicted_coordinate = net(first_spectrogram)
        predicted_coordinate = predicted_coordinate.cpu().numpy()

    def unnormalize(x):
        return (x + 1) / 2 * (_max - _min) + _min

    predicted_coord_unnorm = unnormalize(predicted_coordinate[0])
    actual_coord_unnorm = unnormalize(first_actual_coordinate.cpu().numpy())

    distances.append(np.linalg.norm(predicted_coord_unnorm - actual_coord_unnorm))
print("Predicted Coordinate:", predicted_coord_unnorm)
print("Actual Coordinate:", actual_coord_unnorm)
print("Avg distance error:", np.mean(distances) / 1000, "m")


/tmp/ipykernel_159630/1945945229.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  first_spectrogram = torch.tensor(first_spectrogram, dtype=torch.float32).unsqueeze(0)


Predicted Coordinate: [-2424.72235113 -1899.62980151]
Actual Coordinate: [-2799.34440553  -896.61335945]
Avg distance error: 2.7720396966778496 m
